In [2]:
# from __future__ import print_statement
# import time
# pip install swagger_client
# import swagger_client
# from swagger_client.rest import ApiException
# from pprint import pprint

import pandas as pd
from pandas import json_normalize
import numpy as np
import requests
import json

import ipywidgets as widgets
import spacy
import math
import numpy as np
import scattertext as st
from sklearn import manifold
from sklearn.metrics import euclidean_distances

In [3]:
# pip install json_normalize
# dir(requests.get("https://api.nal.usda.gov/fdc/v1/food/534358?api_key=IDmcqSbD991dUudZIXjztGVAAKnMwhvABtxzlvOQ"))
# food = json.loads(requests.get("https://api.nal.usda.gov/fdc/v1/food/534358?api_key=IDmcqSbD991dUudZIXjztGVAAKnMwhvABtxzlvOQ").content)
# for item in food.items():
#     print(item)

# Direct ID pulls

In [4]:
food = pd.read_csv('complementaryFiles/food.csv')

In [5]:
food.fillna('N/A',inplace=True)

In [6]:
food[(food['description'].str.contains('salmon'))&(food['description'].str.contains('Alaska')&(food['description'].str.contains('sockeye')))]

salmonLst = list(food[food['description'].str.contains('salmon')]['fdc_id'])


In [7]:
# json.loads(requests.get(f'https://api.nal.usda.gov/fdc/v1/foods?&fdcIds=167614,167615&api_key=IDmcqSbD991dUudZIXjztGVAAKnMwhvABtxzlvOQ').content)

In [8]:
salmonLst = str(salmonLst).replace('[','').replace(']','')

In [9]:
apiKey = 'IDmcqSbD991dUudZIXjztGVAAKnMwhvABtxzlvOQ'
# salmon = json.loads(requests.get(f'https://api.nal.usda.gov/fdc/v1/foods?&fdcIds=167614,167615&api_key=IDmcqSbD991dUudZIXjztGVAAKnMwhvABtxzlvOQ').content)
salmon = json.loads(requests.get(f'https://api.nal.usda.gov/fdc/v1/foods?&fdcIds={salmonLst}&api_key={apiKey}').content)

#survey food
# salmon = json.loads(requests.get(f'https://api.nal.usda.gov/fdc/v1/foods?&fdcIds=1098960&api_key=IDmcqSbD991dUudZIXjztGVAAKnMwhvABtxzlvOQ').content)

df = pd.DataFrame(salmon)
df = df.dropna(subset=['foodPortions'])
df = df.reset_index()
df = df.drop('index',axis=1)
df.head()

,fdcId,description,publicationDate,foodNutrients,dataType,foodClass,scientificName,foodComponents,foodAttributes,nutrientConversionFactors,inputFoods,ndbNumber,isHistoricalReference,foodCategory,foodPortions,wweiaFoodCategory,foodCode,startDate,endDate
0,171985,"Fish, salmon, chinook, smoked, (lox), regular",4/1/2019,"[{'nutrient': {'id': 2045, 'number': '951', 'n...",SR Legacy,FinalFood,NaN,[],[],"[{'id': 14494, 'proteinValue': 4.27, 'fatValue...",[],15179.0,True,"{'id': 15, 'code': '1500', 'description': 'Fin...","[{'id': 89683, 'gramWeight': 85.0, 'sequenceNu...",NaN,NaN,NaN,NaN
1,171998,"Fish, salmon, Atlantic, wild, cooked, dry heat",4/1/2019,"[{'nutrient': {'id': 2045, 'number': '951', 'n...",SR Legacy,FinalFood,NaN,[],[],"[{'id': 14507, 'proteinValue': 4.27, 'fatValue...",[],15209.0,True,"{'id': 15, 'code': '1500', 'description': 'Fin...","[{'id': 89708, 'gramWeight': 85.0, 'sequenceNu...",NaN,NaN,NaN,NaN
2,171999,"Fish, salmon, chinook, cooked, dry heat",4/1/2019,"[{'nutrient': {'id': 2045, 'number': '951', 'n...",SR Legacy,FinalFood,NaN,[],[],"[{'id': 14508, 'proteinValue': 4.27, 'fatValue...",[],15210.0,True,"{'id': 15, 'code': '1500', 'description': 'Fin...","[{'id': 89711, 'gramWeight': 154.0, 'sequenceN...",NaN,NaN,NaN,NaN
3,172000,"Fish, salmon, chum, cooked, dry heat",4/1/2019,"[{'nutrient': {'id': 2045, 'number': '951', 'n...",SR Legacy,FinalFood,NaN,[],[],"[{'id': 14509, 'proteinValue': 4.27, 'fatValue...",[],15211.0,True,"{'id': 15, 'code': '1500', 'description': 'Fin...","[{'id': 89713, 'gramWeight': 154.0, 'sequenceN...",NaN,NaN,NaN,NaN
4,172001,"Fish, salmon, pink, cooked, dry heat",4/1/2019,"[{'nutrient': {'id': 2045, 'number': '951', 'n...",SR Legacy,FinalFood,NaN,[],[],"[{'id': 14510, 'proteinValue': 4.27, 'fatValue...",[],15212.0,True,"{'id': 15, 'code': '1500', 'description': 'Fin...","[{'id': 89715, 'gramWeight': 124.0, 'sequenceN...",NaN,NaN,NaN,NaN


In [10]:
current_food = df['description'].values[0]
df['foodNutrients'][0] #hidden dataframe with all of the nutrient data
macros = ['Protein','Carbohydrates','Fat','Total lipid (fat)']
json_normalize(df['nutrientConversionFactors'][0][0])
json_normalize(df['foodCategory'][0])
len(list(json_normalize(df['foodNutrients'][0]).columns))

#legacy food iteration
for x in range(df.shape[0]):
    col_len = len(list(json_normalize(df['foodNutrients'][x]).columns))
    if col_len == 17:
        nutrient_df = json_normalize(df['foodNutrients'][x])
        # display(nutrient_df)
    else:
        pass

#survey food
nutrient_df = json_normalize(df['foodNutrients'][0])
macro_df = nutrient_df[nutrient_df['nutrient.name'].isin(macros)]
conv_factors = json_normalize(df['nutrientConversionFactors'][0])
portion_df = json_normalize(df['foodPortions'][0])
input_df = json_normalize(df['inputFoods'][0])
protein = macro_df[macro_df['nutrient.name']=='Protein']['amount'].values[0]
calories = nutrient_df[nutrient_df['nutrient.unitName']=='kcal']['amount'].values[0]
portion_grams = portion_df['gramWeight'].values[0]
# portion_ounces = portion_df[portion_df['portionDescription'].str.contains('oz')]['portionDescription'].values[0].split()[0]
print(f'Data for {current_food}')
print('===================')
print('===================')
print('Protein per serving')
print(protein)
print('===================')
print('Calories per serving')
print(calories)

Data for Fish, salmon, chinook, smoked, (lox), regular
Protein per serving
18.28
Calories per serving
117.0


# PDF generation

In [11]:
# pip install reportlab

In [12]:
#table gen - https://pythonguides.com/create-and-modify-pdf-file-in-python/

from reportlab.pdfgen import canvas
# c = canvas.Canvas("salmon.pdf")
from reportlab.lib import colors  
from reportlab.lib.pagesizes import letter, inch  
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle   
# # move the origin up and to the left
# c.translate(inch,inch)
# c.setFont("Helvetica", 20)
# c.setFillColorRGB(0,0,0)
# c.drawString(.5*inch, 9*inch, f'Data for {current_food}')
# c.drawString(.5*inch, 8*inch, "========================")
# c.drawString(.5*inch, 7*inch, "========================")
# c.drawString(.5*inch, 6*inch, f'Protein per serving = {protein}')
# c.drawString(.5*inch, 5*inch, "========================")
# c.drawString(.5*inch, 4*inch, f'Calories per serving = {calories}')
# c.drawString(.5*inch, 3*inch, "========================")

# creating a pdf file to add tables  
my_doc = SimpleDocTemplate("salmon.pdf", pagesize = letter)  
my_obj = []  
# defining Data to be stored on table  
my_data = [  
   ["Food", "Calories", "Protein"],  
   [current_food, calories, protein],
]  
# Creating the table with 5 rows  
my_table = Table(my_data, 1 * [2.5 * inch], 2 * [0.5 * inch])  
# setting up style and alignments of borders and grids  
my_table.setStyle(  
   TableStyle(  
       [  
           ("ALIGN", (1, 1), (0, 0), "LEFT"),  
           ("VALIGN", (-1, -1), (-1, -1), "TOP"),  
           ("ALIGN", (-1, -1), (-1, -1), "RIGHT"),  
           ("VALIGN", (-1, -1), (-1, -1), "TOP"),  
           ("INNERGRID", (0, 0), (-1, -1), 1, colors.gray),  
        #    ("BOX", (0, 0), (-1, -1), 2, colors.black),  
       ]  
   )  
)  
my_obj.append(my_table)  
my_doc.build(my_obj)

# c.showPage()
# c.save()

## Search API functionality

In [13]:
cheddar = json.loads(requests.get("https://api.nal.usda.gov/fdc/v1/foods/search?api_key=IDmcqSbD991dUudZIXjztGVAAKnMwhvABtxzlvOQ&query=Cheddar%20Cheese").content)
cheese = json.loads(requests.get("https://api.nal.usda.gov/fdc/v1/foods/search?api_key=IDmcqSbD991dUudZIXjztGVAAKnMwhvABtxzlvOQ&query=Cheese").content)

In [14]:
for key, value in cheese.items():
    if key == 'foods':
        columnHeaders = []
        df=[]
        for val in value:
            keys = []
            values = []
            for key,value in val.items():
                keys.append(key)
                values.append(value)
            tmpKeyDf = pd.DataFrame(keys)
            columnHeaders = list(tmpKeyDf.iloc[:,0].values)
            tmpValueDf = pd.DataFrame(values).T
            tmpValueDf.columns = columnHeaders
            df.append(tmpValueDf)      
            # values = pd.DataFrame(values).T
            # df.append(values)
        valueDf = pd.concat(df)
        display(valueDf.head())
    else:
        pass

,fdcId,description,dataType,gtinUpc,publishedDate,brandOwner,brandName,ingredients,marketCountry,foodCategory,...,foodMeasures,foodAttributes,foodAttributeTypes,foodVersionIds,subbrandName,commonNames,additionalDescriptions,foodCode,foodCategoryId,ndbNumber
0,2383425,CHEESE,Branded,071448504214,2022-12-22,Savencia Cheese USA LLC,SUPREME,"CULTURED PASTEURIZED MILK AND CREAM, SALT, CAL...",United States,Cheese,...,[],[],[],[],NaN,NaN,NaN,NaN,NaN,NaN
0,2442206,CHEESE,Branded,850276005014,2022-12-22,"New Body Nutrition, Inc.",NARTH COAST BEER CHEESE,"CHEDDAR CHEESE, (PASTEURIZED MILK, CHEESE CULT...",United States,Cheese,...,[],[],[],[],NaN,NaN,NaN,NaN,NaN,NaN
0,1943515,CHEESE,Branded,853910006170,2021-07-29,Ferndale Farmstead LLC,FERNDALE FARMSTEAD,"MILK, SALT, CULTURES, ENZYMES.",United States,Cheese,...,[],[],[],[],NaN,NaN,NaN,NaN,NaN,NaN
0,2083541,CHEESE,Branded,039674502084,2021-10-28,American Pride Food Corp.,S.J. FALBO,PARMESAN AND ROMANO CHEESE. MADE FROM PASTEURI...,United States,Cheese,...,[],[],[],[],NaN,NaN,NaN,NaN,NaN,NaN
0,500370,CHEESE,Branded,217260803726,2019-04-01,GOURMET,NaN,"GRUYERE CHEESE AND CHEESE (PASTEURIZED MILK, C...",United States,Cheese,...,[],[],[],[],NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
valueDf['fdcId'].unique()

array([2383425, 2442206, 1943515, 2083541, 500370, 2009474, 2016892,
       2095090, 1917094, 2162106, 2441105, 1986609, 2061880, 1850812,
       1078173, 1913921, 2009472, 2085886, 2089626, 2118226, 2428038,
       2428690, 2469393, 2469939, 2089606, 2008231, 2056312, 2341173,
       2341177, 2341178, 169081, 2341179, 2341182, 2341123, 2341185,
       2341143, 2341190, 2343633, 2341592, 2343095, 2341176, 2341181,
       2341201, 2341112, 2341202, 2341175, 2341167, 2341109, 2341111,
       2341110], dtype=object)

In [16]:
# from pandas.io.json import json_normalize
# nutrients = pd.json_normalize(valueDf['foodNutrients'])
nutrients = []
proteinRatioDfs = []
for fdcId in list(valueDf['fdcId'].values):
    fdcDf = valueDf[valueDf['fdcId']==fdcId]
    for nutrient in fdcDf['foodNutrients'].values:
        nutrientDf = pd.json_normalize(nutrient)
        nutrientDf['fdcId']=fdcId
        proteinRatio = nutrientDf[nutrientDf['nutrientName']=='Protein']['value'].values[0]/nutrientDf[nutrientDf['nutrientName']=='Energy']['value'].values[0]
        # print(proteinRatio)
        df = pd.DataFrame({'fdcId': [fdcId], 'Protein ratio': [proteinRatio]})
        nutrients.append(nutrientDf)
        proteinRatioDfs.append(df)

nutrientDfFull = pd.concat(nutrients)
# nutrientDfFull['fdcId'].unique()
proteinRatioDfFull = pd.concat(proteinRatioDfs)
# proteinRatioDfFull


In [17]:
#### lets use k nearest neighbors on macros !!!! ######
proteinRatioDfFull

,fdcId,Protein ratio
0,2383425,0.045504
0,2442206,0.044548
0,1943515,0.063746
0,2083541,0.120000
0,500370,0.073502
0,2009474,0.071600
0,2016892,0.063613
0,2095090,0.081679
0,1917094,0.058275
0,2162106,0.050000
